In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.data.external import *
from ipywidgets import *
import pandas as pd
import numpy as np
import math
import sqlite3
from random import randint

In [96]:
# Local stuff
from api.helpers import *
import api.tab_model as tm
import api.imageclassifier as ic
from api.location_model import LocationModel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
img_learn = ic.ImageClassifier('models/v0.4/resnet101-f7-fp16-h1.0.pkl')

In [97]:
tab_learn = tm.TabModel('dbs/fungid.sqlite')
loc_learn = LocationModel('dbs/fungid.sqlite')

In [94]:
file = "dbs/training/training-data-v0-4.csv"
data = pd.read_csv(file)
data = data.loc[data.imgid.notna()].copy()
data['img'] = 'dbs/images/500/' + data.gbifid.astype(str) + '-' + data.imgid.astype(int).astype(str) + '.png'

In [13]:
def get_bounding_box(lat, lon, dist):
    latdiff = (180 / math.pi) * (dist / 6378137)
    londiff = (180 / math.pi) * (dist / 6378137) / math.cos(lat)
    return (lat - latdiff, lon - londiff), (lat + latdiff, lon + londiff)

def get_db_species(conn, observation, dist):
    p1, p2 = get_bounding_box(tab_item.decimallatitude, tab_item.decimallongitude, dist)
    print(p1, p2)
    cursor = conn.execute("""
    SELECT species, COUNT(*)
    FROM validobservations v 
    JOIN trainingspecies t ON v.specieskey = t.specieskey
    WHERE decimallatitude BETWEEN ? AND ? 
    AND decimallongitude BETWEEN ? AND ? 
    GROUP BY 1 ORDER BY 2;""",
                 (p1[0], p2[0], p1[1], p2[1]))
    results = cursor.fetchall()
    print(len(results))
    
def get_tab_data(data):
    tab_columns = ['kg', 'elu_class1', 'elu_class2', 'elu_class3', 'decimallatitude', 'decimallongitude', 'species', 'normalizedmonth', 'season']
    return data[tab_columns].copy()

def get_results(learner, data):
    row, clas, probs = learner.predict(data)
    return probs
                 


In [160]:


# loc_learn.get_predictions(46.7867, -92.1005)


0.22458364500267453 0.5586388734291332
(52.025416354997326, 5.591361126570867) (52.474583645002674, 6.708638873429134)


,total,local
species,,
Abortiporus biennis,3771,88
Acarospora fuscata,1667,108
Acarospora moenium,254,39
Acrospermum compressum,776,13
Agaricus arvensis,5292,35
...,...,...
Xylaria oxyacanthae,196,3
Xylaria polymorpha,10333,143
Xylodon flaviporus,761,16


In [292]:
def compare(probs, vals, num) -> pd.DataFrame:

    df = pd.DataFrame()
    for i, prob in enumerate(probs):
        if i == 0:
            df["prob" + str(i)] = prob
            df["rank" + str(i)] = prob.rank(ascending=False)
        else: 
            df = pd.concat([df, prob, prob.rank(ascending=False)], axis=1)

    df = pd.concat([df] + vals, axis=1)

    return df.sort_values(by="rank0").head(num)

# 124978
# 2119839 - Bad
# 1292929 - Negative nubmers = 
# 454673 - Dropped to 10 from 1
# 1068383 - Artificial fucks with stuff. 
# 193834 & 2073568- Totall throws the data into mayhem 
# 1370890 - Ignore Surface Water in ELU?
# Need to only pull 4-5 images per obs. 
# 
id = randint(0, len(img_data))
print(id)
item = data.iloc[id]
print(item)
images = list(data.loc[data.gbifid == item.gbifid].img)
print(images)
obs = obs_from_series(item, images)

loc_preds = loc_learn.get_predictions(obs.lat, obs.long)
tab_preds = tab_learn.get_predictions(obs)
print((tab_preds.max(), tab_preds.min(), tab_preds.loc[tab_item.species]))

img_preds, img_df = img_learn.get_predictions(obs.images)

df = compare([img_preds, 
              img_preds * tab_preds,
              (img_preds * tab_preds).loc[loc_preds.index]
             ], [tab_preds], 50)
df

# 2238561578

1230473
gbifid                                   3415500486
_family                                Bolbitiaceae
genus                                      Conocybe
species                              Conocybe apala
familykey                                    8440.0
genuskey                                    2529714
specieskey                                  2529827
eventyear                                      2020
eventmonth                                        9
eventday                                         11
eventdate                       2020-09-11 09:50:23
decimallatitude                           34.005727
decimallongitude                        -117.648225
kg                                                8
elu                                             222
elu_class1                 Artificial or Urban Area
elu_class2                                      NaN
elu_class3                                      NaN
normalizedmonth                                  10
seas

,prob0,rank0,0,1,0,1,likelihood
species,,,,,,,
Conocybe apala,21.723814,1.0,18.556285,1.0,18.556285,1.0,0.854191
Parasola plicatilis,17.338703,2.0,13.809979,2.0,13.809979,2.0,0.796483
Parasola auricoma,14.914367,3.0,10.861019,7.0,10.861019,6.0,0.728225
Parasola lactea,14.325360,4.0,12.050973,4.0,NaN,NaN,0.841234
Panaeolina foenisecii,13.992872,5.0,12.274992,3.0,12.274992,3.0,0.877232
Coprinellus domesticus,13.513763,6.0,11.140368,5.0,11.140368,4.0,0.824372
Bolbitius titubans,13.375378,7.0,10.904797,6.0,10.904797,5.0,0.815289
Coprinellus disseminatus,12.964526,8.0,10.662212,8.0,10.662212,7.0,0.822414
Coprinellus micaceus,12.562638,9.0,9.805508,13.0,9.805508,9.0,0.780529


In [261]:
img_df.sort_values(by="0", ascending=False)

,0,1
species,,
Panus conchatus,21.042353,16.658409
Pleurotus pulmonarius,16.273634,16.519880
Pleurotus ostreatus,16.180414,15.065679
Lenzites betulinus,15.551631,16.919554
Panus neostrigosus,14.042208,14.673512
...,...,...
Strobilomyces strobilaceus,0.408175,3.857002
Suillus americanus,0.362640,5.565790
Chlorophyllum olivieri,0.202737,6.143771


In [205]:
probs = [img_preds, 
         img_preds * tab_preds,
         (img_preds * tab_preds).loc[loc_preds.index]
        ]

df = pd.DataFrame()
for i, prob in enumerate(probs):
    if i == 0:
        df["prob" + str(i)] = prob
        df["rank" + str(i)] = prob.rank(ascending=False)
    else: 
        df = pd.concat([df, prob, prob.rank(ascending=False)], axis=1)

    
df
# (img_preds * tab_preds).loc[loc_preds.index]

,prob0,rank0,0,1,0,1
species,,,,,,
Abortiporus biennis,-11.112320,2767.0,-17.522710,2240.0,-17.522710,1319.0
Acanthophysium oakesii,-6.217802,1092.0,-15.527212,1985.0,NaN,NaN
Acarospora fuscata,-5.913771,900.0,-17.303498,2211.0,-17.303498,1300.0
Acarospora moenium,-5.424222,671.0,-14.617180,1850.0,-14.617180,1060.0
Acarospora sinopica,-7.454016,1756.0,-17.666347,2260.0,-17.666347,1332.0
...,...,...,...,...,...,...
Xylodon flaviporus,-8.361444,2177.0,-20.473418,2528.0,NaN,NaN
Xylodon radula,-6.460116,1219.0,-15.530566,1986.0,-15.530566,1146.0
Xylodon sambuci,-10.604612,2721.0,-21.413704,2595.0,-21.413704,1554.0


In [306]:
image = PILImage.create('/Users/weishuhn/Downloads/1920px-Galerina_marginata_Point_Reyes.jpg')
# img_learn.resize(image)
pred, _ = img_learn.get_predictions([image])
compare([pred], [], 50)

,prob0,rank0
species,,
Galerina marginata,23.240204,1.0
Conocybe aporos,21.501135,2.0
Galerina patagonica,20.226049,3.0
Psilocybe ovoideocystidiata,19.769825,4.0
Agrocybe praecox,19.472174,5.0
Cyclocybe erebia,19.415812,6.0
Psathyrella piluliformis,19.414293,7.0
Suillus flavidus,19.325062,8.0
Armillaria mellea,18.983782,9.0


In [219]:
type(df.sum(axis=1))

pandas.core.series.Series

In [ ]:
url = 'https://inaturalist-open-data.s3.amazonaws.com/photos/3897840/large.jpg'
filename = 'tmp/tmp.jpg'
resized = 'tmp/tmp-resized.jpg'
resize = Resize(224, ResizeMethod.Pad, pad_mode='zeros')
download_url(url, filename)
img = PILImage.create(filename)
resize(img).save(resized)
test_probs = get_results(img_learn, resized)
print_top50(test_probs)